# Setfit few shot learning

In [ ]:
#!pip install setfit

In [4]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer

## Load custom dataset

In [6]:
dataset = load_dataset('csv', data_files='data/smabuse.csv', split='train')
print(dataset)

Using custom data configuration default-1919447a6b1dcd37


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/gary_hutson/.cache/huggingface/datasets/csv/default-1919447a6b1dcd37/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


/home/gary_hutson/anaconda3/envs/transformers/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:727: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


# Push our custom dataset to the hub

In [ ]:
padd_len = 43
iterate = 10

for idx, i in enumerate(dataset['train']):
    # Print the text
    print('-' * padd_len, '[TEXT]', '-' * padd_len)
    print(dataset['train']['text'][idx])
    # Print the label
    print('-' * padd_len, '[LABEL]', '-' * padd_len)
    print(dataset['train']['label'][idx])

    # Break the print after n iterations
    if idx > iterate:
        break


## Simulate few shot learning by sampling 8 examples per class

In [ ]:
num_class = 2
train_ds = dataset['train'].shuffle(seed=123).select(range(8 * num_class))
eval_ds = dataset['test']

In [ ]:
train_ds['text']

## Load a model from hub

In [ ]:
model = SetFitModel.from_pretrained('sentence-transformers/paraphrase-mpnet-base-v2')

## Create Trainer

In [ ]:
trainer = SetFitTrainer(
    model=model, 
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    loss_class=CosineSimilarityLoss, 
    batch_size=16,
    num_iterations=20,# The number of text pairs to generate for contrastive learning
    column_mapping={"text": "text", "label": "label"}
)

In [ ]:
trainer.train()

## Evaluate the model

In [ ]:
metrics = trainer.evaluate()

In [ ]:
print(metrics)

## Publish model to hub

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
!huggingface-cli whoami

In [ ]:
!git config --global credential.helper store

## Push the model to the hub

In [ ]:
MODEL_NAME = 'StatsGary/setfit-ft-sentinent-eval'
trainer.push_to_hub(repo_path_or_name=MODEL_NAME)

## Load model from Hub for inference

In [ ]:
model_infer = SetFitModel.from_pretrained(MODEL_NAME)

In [ ]:
preds = model_infer(["i loved the spiderman movie!", "I hate white people"]) 

In [ ]:
print(preds)

## Access via endpoint

Skip the below if you want to perform local inference. 

In [ ]:
import json
import requests as r

ENDPOINT_URL=""# url of your endpoint
HF_TOKEN="" #Your HF token

# payload samples
regular_payload = { "inputs": "Coming to The Rescue Got a unique problem? Not to worry: you can find a financial planner for every specialized need"}

# HTTP headers for authorization
headers= {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

# send request
response = r.post(ENDPOINT_URL, headers=headers, json=paramter_payload)
classified = response.json()

print(classified)